# 📊 Prophet Forecast — Website Hourly Sessions (Google Analytics)
This notebook uses **Google Analytics public dataset** to forecast hourly website sessions. Demonstrates production-grade Prophet modeling with data quality checks, extended training window, and rigorous validation methodology.

**Data Source:** BigQuery Public Dataset (bigquery-public-data.google_analytics_sample)

**Improvements over baseline:**
- ✅ Extended training window (12 months for yearly seasonality)
- ✅ Comprehensive data quality checks (missing hours, duplicates, outliers)
- ✅ 80/20 holdout validation with multiple metrics
- ✅ Linear growth (appropriate for unbounded web traffic)
- ✅ Optional hyperparameter tuning (50 parameter combinations tested)
- ✅ Uncertainty quantification with 90% confidence intervals

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as _stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from prophet import Prophet
from prophet.plot import plot_plotly
from prophet.diagnostics import cross_validation, performance_metrics
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

print("="*70)
print("PROPHET FORECAST — WEBSITE HOURLY SESSIONS (GOOGLE ANALYTICS)")
print("="*70)

## ⚙️  Configuration

In [ ]:
# CONFIG
TRAINING_MONTHS = 12      
RUN_TUNING = False        # Toggle cross-validation + grid search

# Forecast period options:
FORECAST_DAYS = 7        # ← Change this: 7 (1 week), 14 (2 weeks), 28 (4 weeks), etc.
FORECAST_HOURS = 24 * FORECAST_DAYS  # Automatically converts days to hours

TEST_SIZE = 0.2           # Holdout validation (20% of data)

weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_colors = {
    'Monday': '#1f77b4', 'Tuesday': '#ff7f0e', 'Wednesday': '#2ca02c',
    'Thursday': '#d62728', 'Friday': '#9467bd', 'Saturday': '#8c564b', 'Sunday': '#e377c2'
}

print(f"📋 Configuration:")
print(f"   Training: {TRAINING_MONTHS} months")
print(f"   Forecast: {FORECAST_DAYS} days ({FORECAST_HOURS} hours)")
print(f"   Tuning: {'Enabled' if RUN_TUNING else 'Disabled'}")

## 1️⃣ Fetch Data from BigQuery

In [ ]:
print("\n📊 Step 1: Fetching data from BigQuery...")

# FREE PUBLIC DATASET: Google Analytics Sample (from BigQuery public datasets)
# https://console.cloud.google.com/marketplace/product/google/analytics-sample-dataset
# NOTE: This dataset contains data from Aug 1, 2016 to Aug 1, 2017 only

# Calculate how many months to fetch (max 12 months from the available data)
# Dataset spans: 2016-08-01 to 2017-08-01
if TRAINING_MONTHS <= 12:
    # Use the LAST N months of available data (ending Aug 1, 2017)
    query = f"""
    SELECT
      TIMESTAMP_TRUNC(
        PARSE_TIMESTAMP('%s', CAST(visitStartTime AS STRING)), 
        HOUR
      ) AS ts_hour,
      FORMAT_TIMESTAMP('%A', PARSE_TIMESTAMP('%s', CAST(visitStartTime AS STRING))) AS weekday,
      EXTRACT(HOUR FROM PARSE_TIMESTAMP('%s', CAST(visitStartTime AS STRING))) AS hour,
      COUNT(DISTINCT fullVisitorId) AS session_count
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE _TABLE_SUFFIX BETWEEN 
      FORMAT_DATE('%Y%m%d', DATE_SUB(DATE('2017-08-01'), INTERVAL {TRAINING_MONTHS} MONTH))
      AND '20170801'
    GROUP BY ts_hour, weekday, hour
    ORDER BY ts_hour;
    """
else:
    # If requesting more than 12 months, use full dataset (12 months)
    print(f"   ⚠️  Requested {TRAINING_MONTHS} months, but dataset only has 12 months")
    print(f"   Using full dataset: Aug 1, 2016 to Aug 1, 2017")
    query = """
    SELECT
      TIMESTAMP_TRUNC(
        PARSE_TIMESTAMP('%s', CAST(visitStartTime AS STRING)), 
        HOUR
      ) AS ts_hour,
      FORMAT_TIMESTAMP('%A', PARSE_TIMESTAMP('%s', CAST(visitStartTime AS STRING))) AS weekday,
      EXTRACT(HOUR FROM PARSE_TIMESTAMP('%s', CAST(visitStartTime AS STRING))) AS hour,
      COUNT(DISTINCT fullVisitorId) AS session_count
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    GROUP BY ts_hour, weekday, hour
    ORDER BY ts_hour;
    """

# Use your GCP project ID
df = pd.read_gbq(query, project_id="mythic-code-477217-a8", dialect="standard")

print(f"   ℹ️  Using historical data from Google Analytics Sample dataset (2016-2017)")
print(f"   Note: This is historical data for demonstration purposes")

print("✅ Data fetched successfully!")
print(f"   Rows: {len(df)} | Columns: {list(df.columns)}")

# Safety check for empty dataframe
if len(df) == 0:
    raise ValueError(
        "❌ Query returned 0 rows! Possible issues:\n"
        "   1. Date range doesn't match available data (GA Sample: 2016-08-01 to 2017-08-01)\n"
        "   2. Project ID may not have access to public datasets\n"
        "   3. Table name or syntax error in query\n"
        "\n   Try running this test query in BigQuery console:\n"
        "   SELECT COUNT(*) FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`"
    )

## 2️⃣ Data Quality Checks & Preparation

In [ ]:
print("\n" + "="*70)
print("DATA QUALITY CHECKS & PREPARATION")
print("="*70)

# Basic renaming and type conversion
df = df.rename(columns={'ts_hour': 'date', 'session_count': 'count'})
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)
df['hour'] = df['hour'].astype(int)
df['count'] = df['count'].astype(float)

print(f"\n📅 Initial data range: {df['date'].min()} → {df['date'].max()}")
print(f"   Total hours in query: {len(df)}")

# ========================================
# NEW: DATA QUALITY CHECKS
# ========================================

print("\n🔍 Data Quality Checks:")

# 1. Check for duplicates
dupes = df[df.duplicated(subset=['date'], keep=False)]
if len(dupes) > 0:
    print(f"⚠️  Found {len(dupes)} duplicate timestamps - removing...")
    df = df.drop_duplicates(subset=['date'], keep='first')
else:
    print("✅ No duplicate timestamps found")

# 2. Check for missing hours
expected_hours = pd.date_range(
    start=df['date'].min(),
    end=df['date'].max(),
    freq='H'
)
expected_count = len(expected_hours)
actual_count = len(df)
missing_count = expected_count - actual_count

if missing_count > 0:
    print(f"⚠️  Found {missing_count} missing hours ({missing_count/expected_count:.2%}) - filling with 0")
    
    # Create complete hourly sequence
    df_complete = pd.DataFrame({'date': expected_hours})
    df = df_complete.merge(df, on='date', how='left')
    
    # Fill missing values
    df['count'] = df['count'].fillna(0)
    df['hour'] = df['date'].dt.hour
    df['weekday'] = df['date'].dt.day_name()
else:
    print("✅ No missing hours detected - continuous time series")

# 3. Outlier detection (flag, don't remove)
q1, q3 = df['count'].quantile([0.25, 0.75])
iqr = q3 - q1
outlier_threshold = q3 + 3 * iqr
outliers = df[df['count'] > outlier_threshold]

if len(outliers) > 0:
    print(f"ℹ️  Flagged {len(outliers)} potential outliers (>{outlier_threshold:.0f} sessions/hour)")
    max_idx = outliers['count'].idxmax()
    print(f"   Max: {outliers.loc[max_idx, 'count']:.0f} sessions at {outliers.loc[max_idx, 'date']}")
    print(f"   Note: Keeping outliers in model (Prophet is robust)")
else:
    print("✅ No extreme outliers detected")

# 4. Check for zero-inflation
zero_share = (df['count'] == 0).mean()
print(f"\nℹ️  Share of zero-hour values: {zero_share:.2%}")
if zero_share > 0.1:
    print(f"   ⚠️  High zero proportion - using linear growth (appropriate for web traffic)")

print("\n✅ Data quality checks complete")
print(f"   Final dataset: {len(df)} hours from {df['date'].min()} to {df['date'].max()}")

# Prepare Prophet data
df_prophet = df[['date', 'count']].rename(
    columns={'date': 'ds', 'count': 'y'}
).sort_values('ds').reset_index(drop=True)

print("\n📊 Prophet data summary:")
print(df_prophet['y'].describe())

## 3️⃣ Train/Test Split

**Validation Strategy:**
- **Single chronological holdout split** (80% train / 20% test)
- Best practice for time series: train on past → test on future
- Avoids data leakage and mimics real-world forecasting
- Cross-validation (multiple rolling splits) happens during tuning only

In [ ]:
print("\n" + "="*70)
print("TRAIN/TEST SPLIT FOR VALIDATION")
print("="*70)

# IMPORTANT: Single holdout split (chronological)
# This is BEST PRACTICE for time series forecasting:
# - Uses first 80% for training, last 20% for testing (chronological order)
# - Simulates real-world scenario: train on past, predict future
# - Avoids data leakage (never train on future data to predict past)
# - Single split is appropriate because time series data is ordered
#
# NOTE: Cross-validation (multiple rolling splits) happens during TUNING only
# when RUN_TUNING=True. Tuning uses 8-10 rolling windows to select best parameters.
# This holdout test provides final unbiased evaluation of the chosen model.

# Split: 80% train, 20% test
split_idx = int(len(df_prophet) * (1 - TEST_SIZE))
df_train = df_prophet.iloc[:split_idx].copy()  # First 80% (chronological)
df_test = df_prophet.iloc[split_idx:].copy()   # Last 20% (future)

print(f"\n🔀 Train/Test Split ({int((1-TEST_SIZE)*100)}% / {int(TEST_SIZE*100)}%):")
print(f"   Train: {df_train['ds'].min()} to {df_train['ds'].max()}")
print(f"          {len(df_train)} hours ({len(df_train)/24:.1f} days)")
print(f"   Test:  {df_test['ds'].min()} to {df_test['ds'].max()}")
print(f"          {len(df_test)} hours ({len(df_test)/24:.1f} days)")
print(f"\n   Train mean: {df_train['y'].mean():.2f} sessions/hour")
print(f"   Test mean:  {df_test['y'].mean():.2f} sessions/hour")

## 4️⃣ Hyperparameter Tuning (Optional)

In [ ]:
def tune_prophet(
    df_prophet,
    param_grid=None,
    initial='90 days',
    period='30 days',
    horizon='7 days',      
    base_kwargs=None
):
    """Cross-validation + grid search for Prophet. Returns best_model, results_df."""
    import itertools

    if param_grid is None:
        # IMPROVED: Wider parameter ranges
        param_grid = {
            'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1, 0.5],
            'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0, 20.0],
            'seasonality_mode': ['additive', 'multiplicative']
        }
    if base_kwargs is None:
        base_kwargs = dict(
            growth='linear',              # Linear for unbounded web traffic
            daily_seasonality=True,
            weekly_seasonality=True,
            yearly_seasonality=True,
            interval_width=0.90
        )

    combos = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    results = []
    print(f"\n🔍 Tuning Prophet: testing {len(combos)} parameter combinations...")
    
    for i, params in enumerate(combos, 1):
        try:
            if i % 10 == 0:
                print(f"   Progress: {i}/{len(combos)} combinations tested...")
            
            kwargs = dict(base_kwargs)
            kwargs.update(params)
            
            # Prepare data (no cap/floor needed for linear growth)
            df_tune = df_prophet.copy()
            
            m = Prophet(**kwargs).fit(df_tune)
            df_cv = cross_validation(m, initial=initial, period=period, horizon=horizon)
            df_p = performance_metrics(df_cv)
            
            results.append({
                **params,
                'rmse': float(df_p['rmse'].mean()),
                'mae': float(df_p['mae'].mean()),
                'mape': float(df_p['mape'].mean()),
                'coverage': float(df_p['coverage'].mean())
            })
        except Exception as e:
            print(f"⚠️ Failed for params {params}: {e}")

    res = pd.DataFrame(results).sort_values('rmse')
    if res.empty:
        raise RuntimeError("No tuning results; check data length / CV windows.")
    
    best_params = res.iloc[0][list(param_grid.keys())].to_dict()
    print("\n✅ Best parameters (by RMSE):")
    for k, v in best_params.items():
        print(f"   {k}: {v}")
    print(f"   Best RMSE: {res.iloc[0]['rmse']:.2f}")
    print(f"   Best MAE: {res.iloc[0]['mae']:.2f}")
    print(f"   Best MAPE: {res.iloc[0]['mape']:.2%}")
    
    best_kwargs = dict(base_kwargs)
    best_kwargs.update(best_params)
    
    # Return best parameters only (model will be trained in next cell)
    return best_kwargs, res

## 5️⃣ Train Prophet Model

In [ ]:
print("\n" + "="*70)
print("TRAINING PROPHET MODEL")
print("="*70)

# CALIBRATED PARAMETERS - Optimized defaults based on tuning results
# Note: These will be overridden if RUN_TUNING=True
base_kwargs = dict(
    growth='linear',                    # Linear for unbounded web traffic
    changepoint_prior_scale=0.05,       # Moderate trend flexibility (tuning optimal)
    seasonality_prior_scale=0.01,       # Low seasonality strength (tuning optimal)
    seasonality_mode='multiplicative',  # Percentage-based patterns (tuning optimal)
    daily_seasonality=True,             # Essential for hourly data
    weekly_seasonality=True,            # Essential for business patterns
    yearly_seasonality=True,            # Enabled with 12 months data
    interval_width=0.90                 # 90% confidence intervals
)

# No cap/floor needed for linear growth
if RUN_TUNING:
    print("\n🧪 Running hyperparameter tuning...")
    print("   ⚠️  This may take 5-15 minutes (testing 50 parameter combinations)")
    print("   Progress bars show Prophet's internal MCMC sampling for each combination\n")
    
    best_params, tuning_results = tune_prophet(
        df_train,
        param_grid=None,
        initial='90 days',      # 25% of 12 months training data
        period='30 days',       # Monthly validation cutoffs
        horizon='7 days',      # Match FORECAST_DAYS config
        base_kwargs=base_kwargs
    )
    
    # Update base_kwargs with best parameters from tuning
    base_kwargs.update(best_params)
    
    print(f"\n✅ Updated parameters after tuning:")
    print(f"   Changepoint prior: {best_params['changepoint_prior_scale']}")
    print(f"   Seasonality prior: {best_params['seasonality_prior_scale']}")
    print(f"   Seasonality mode: {best_params['seasonality_mode']}")
    
    # Display top 10 results
    print("\n📊 Tuning Summary (Top 10 by RMSE):")
    cols = ['changepoint_prior_scale', 'seasonality_prior_scale', 'seasonality_mode', 
            'rmse', 'mae', 'mape', 'coverage']
    print(tuning_results[cols].head(10).to_string(index=False))

# Train final model with best parameters (or defaults if tuning disabled)
print("\n📊 Training final Prophet model...")
model = Prophet(**base_kwargs)
model.fit(df_train)

print("\n✅ Prophet model trained successfully!")

# Show model parameters
print("\n📋 Model Configuration:")
print(f"   Growth: {model.growth}")
print(f"   Changepoint prior scale: {model.changepoint_prior_scale}")
print(f"   Seasonality prior scale: {model.seasonality_prior_scale}")
print(f"   Seasonality mode: {model.seasonality_mode}")
print(f"   Daily seasonality: {model.daily_seasonality}")
print(f"   Weekly seasonality: {model.weekly_seasonality}")
print(f"   Yearly seasonality: {model.yearly_seasonality}")

print("\n" + "="*70)
print("🎯 PARAMETER CALIBRATION EXPLAINED")
print("="*70)

print(f"""
ACTUAL MODEL CONFIGURATION (as trained):
   Growth: {model.growth}
   Seasonality mode: {model.seasonality_mode}
   Changepoint prior: {model.changepoint_prior_scale}
   Seasonality prior: {model.seasonality_prior_scale}

1️⃣ GROWTH MODEL: Linear 
   Why: Web traffic has NO UPPER BOUND (can grow indefinitely)
   ✅ Linear: Trend continues unbounded → y = intercept + slope*t
   
   Use Linear when:
   - Growth can continue indefinitely (web traffic, cloud usage)
   - No natural ceiling exists
   - Early growth stage (pre-saturation)

2️⃣ SEASONALITY MODE: {model.seasonality_mode.upper()}
   This was {"selected by tuning" if RUN_TUNING else "set as default"}
   
   • Additive: Seasonality = fixed amount (e.g., +50 sessions at peak)
     Best for: Sparse data, many zeros, constant seasonal patterns
   
   • Multiplicative: Seasonality = percentage (e.g., +20% at peak)
     Best for: Dense data, seasonal patterns scale with level

3️⃣ CHANGEPOINT_PRIOR_SCALE: {model.changepoint_prior_scale}
   Controls trend flexibility
   - Lower (0.001-0.01): Rigid trend → underfits
   - Moderate (0.05-0.1): Balanced → recommended
   - Higher (0.5+): Flexible trend → may overfit
   
4️⃣ SEASONALITY_PRIOR_SCALE: {model.seasonality_prior_scale}
   Controls seasonal pattern strength
   - Lower (0.01-0.1): Weak seasonality
   - Moderate (1.0-10.0): Normal seasonality
   - Higher (20.0+): Strong seasonality

5️⃣ YEARLY_SEASONALITY: Enabled (12 months data)
   Captures annual patterns (holidays, seasonal trends)
   Requires ≥12 months data (2+ years ideal)

✅ VALIDATION STRATEGY:
   - 80/20 holdout split (chronological)
   - Multiple metrics (RMSE, MAE, MAPE, R², coverage, bias)
   - Residual analysis (Q-Q plot, heteroscedasticity check)
   - Cross-validation with tuning (RUN_TUNING=True) tests 50 combinations

🎯 RESULT: Production-ready forecasting system
""")

## 6️⃣ Holdout Validation

In [ ]:
print("\n" + "="*70)
print("HOLDOUT VALIDATION ON TEST SET")
print("="*70)

print(f"\n🤖 Model Used for Validation:")
print(f"   Seasonality mode: {model.seasonality_mode}")
print(f"   Changepoint prior: {model.changepoint_prior_scale}")
print(f"   Seasonality prior: {model.seasonality_prior_scale}")

# Prepare test set
df_test_pred = df_test.copy()

# Predict on test set
forecast_test = model.predict(df_test_pred)
forecast_test['yhat'] = forecast_test['yhat'].clip(lower=0)

# Calculate metrics
y_true = df_test['y'].values
y_pred = forecast_test['yhat'].values

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true == 0, 1, y_true))) * 100
r2 = r2_score(y_true, y_pred)
bias = np.mean(y_pred - y_true)

# Coverage
in_interval = ((y_true >= forecast_test['yhat_lower'].values) & 
               (y_true <= forecast_test['yhat_upper'].values))
coverage = in_interval.mean()

print("\n📊 Test Set Performance:")
print(f"   RMSE:     {rmse:.2f} sessions/hour")
print(f"   MAE:      {mae:.2f} sessions/hour")
print(f"   MAPE:     {mape:.2f}%")
print(f"   R²:       {r2:.3f}")
print(f"   Bias:     {bias:.2f} sessions/hour (should be ~0)")
print(f"   Coverage: {coverage:.1%} (target: 90%)")

# Interpretation
print("\n📈 Performance Interpretation:")
if mape < 10:
    print("   ✅ Excellent: MAPE < 10%")
elif mape < 20:
    print("   ✅ Good: MAPE 10-20%")
elif mape < 30:
    print("   ⚠️  Fair: MAPE 20-30%")
else:
    print("   ❌ Poor: MAPE > 30%")

if 0.85 <= coverage <= 0.95:
    print("   ✅ Coverage within expected range (85-95%)")
elif coverage < 0.85:
    print("   ⚠️  Coverage too low - model underestimates uncertainty")
else:
    print("   ⚠️  Coverage too high - model overestimates uncertainty")

if abs(bias) < mae * 0.1:
    print("   ✅ Low bias - model is well-calibrated")
else:
    if bias > 0:
        print("   ⚠️  Positive bias - model tends to overpredict")
    else:
        print("   ⚠️  Negative bias - model tends to underpredict")

In [ ]:
# 🔍 DIAGNOSTIC: Check data characteristics
print("\n" + "="*70)
print("DATA DIAGNOSTICS")
print("="*70)

print("\n📊 Training Data Statistics:")
print(f"   Mean: {df_train['y'].mean():.2f}")
print(f"   Median: {df_train['y'].median():.2f}")
print(f"   Min: {df_train['y'].min():.2f}")
print(f"   Max: {df_train['y'].max():.2f}")
print(f"   Std: {df_train['y'].std():.2f}")
print(f"   Zeros: {(df_train['y'] == 0).sum()} ({(df_train['y'] == 0).mean():.1%})")

print("\n📊 Test Data Statistics:")
print(f"   Mean: {df_test['y'].mean():.2f}")
print(f"   Median: {df_test['y'].median():.2f}")
print(f"   Min: {df_test['y'].min():.2f}")
print(f"   Max: {df_test['y'].max():.2f}")

print("\n🔍 Sample predictions vs actuals (first 24 hours):")
sample = forecast_test[['ds', 'yhat']].head(24).copy()
sample['actual'] = df_test['y'].values[:24]
sample['error'] = sample['yhat'] - sample['actual']
print(sample[['ds', 'actual', 'yhat', 'error']].to_string(index=False))

print("\n DIAGNOSIS:")
if df_train['y'].mean() < 10:
    print("   ❌ Very sparse data (mean < 10) - hourly aggregation may be too granular")
    print("   💡 Recommendation: Aggregate to daily level instead of hourly")
elif (df_train['y'] == 0).mean() > 0.3:
    print("   ❌ High zero-inflation (>30%) - multiplicative seasonality will fail")
    print("   💡 Recommendation: Switch to additive seasonality")
elif df_train['y'].std() / df_train['y'].mean() > 2:
    print("   ⚠️  Very high variance - data may need log transformation")
    print("   💡 Recommendation: Use log(y+1) transformation")
else: 
    print("Parameters in range. The current setup is reflecting the dataset characteristics.")

## 7️⃣ Forecast Next 7 Days

In [ ]:
print("\n" + "="*70)
print("FORECASTING NEXT 7 DAYS")
print("="*70)

# Retrain on full dataset for production forecast
print("\n🔄 Retraining on full dataset for production forecast...")
df_full = df_prophet.copy()

model_full = Prophet(**base_kwargs).fit(df_full)

# Generate forecast (no cap/floor for linear growth)
future = model_full.make_future_dataframe(periods=FORECAST_HOURS, freq='H')

forecast = model_full.predict(future)
print("✅ Forecast generated.")

# Clip negatives
forecast['yhat'] = forecast['yhat'].clip(lower=0)
forecast['yhat_lower'] = forecast['yhat_lower'].clip(lower=0)
forecast['yhat_upper'] = forecast['yhat_upper'].clip(lower=0)

# Extract future forecast
forecast_future = forecast.tail(FORECAST_HOURS).copy()
forecast_future['weekday'] = forecast_future['ds'].dt.day_name()
forecast_future['hour'] = forecast_future['ds'].dt.hour

# IMPROVED: Aggregate using MEAN (not first)
prophet_agg = (
    forecast_future
    .groupby(['weekday','hour'], as_index=False)
    .agg({
        'yhat': 'mean',
        'yhat_lower': 'mean',
        'yhat_upper': 'mean'
    })
    .rename(columns={
        'yhat': 'prophet_mean',
        'yhat_lower': 'prophet_lower',
        'yhat_upper': 'prophet_upper'
    })
)

# Add standard deviation for uncertainty
prophet_std = (
    forecast_future
    .groupby(['weekday','hour'], as_index=False)['yhat']
    .std()
    .rename(columns={'yhat': 'prophet_std'})
)
prophet_agg = prophet_agg.merge(prophet_std, on=['weekday', 'hour'])

prophet_agg['weekday'] = pd.Categorical(prophet_agg['weekday'], categories=weekday_order, ordered=True)
prophet_agg = prophet_agg.sort_values(['weekday','hour']).reset_index(drop=True)

print(f"\n📊 Summary for next {FORECAST_DAYS} days (representative week):")
print(f"   Avg hourly forecast: {prophet_agg['prophet_mean'].mean():.2f} sessions/hour")
print(f"   Avg uncertainty (std): {prophet_agg['prophet_std'].mean():.2f} sessions/hour")

daily_totals = prophet_agg.groupby('weekday')['prophet_mean'].sum().reindex(weekday_order)
print("\n   Daily totals by weekday:")
for day, total in daily_totals.items():
    print(f"      {day}: {total:.1f} sessions")

## 8️⃣ Visualizations (Static)

In [ ]:
print("\n" + "="*70)
print("VISUALIZATIONS (STATIC)")
print("="*70)

# Heatmaps: Prophet mean and CI width
fig, axes = plt.subplots(1, 2, figsize=(22, 8))

heatmap_mean = prophet_agg.pivot(index='hour', columns='weekday', values='prophet_mean').reindex(columns=weekday_order)
sns.heatmap(heatmap_mean, cmap='YlOrRd', annot=True, fmt='.1f',
            cbar_kws={'label': 'Sessions/Hour'}, linewidths=0.5, ax=axes[0])
axes[0].set_title("Prophet Forecast (Hourly Mean)", fontsize=13, fontweight='bold')
axes[0].set_xlabel("Weekday"); axes[0].set_ylabel("Hour (UTC)")

heatmap_ci = (prophet_agg.assign(ci_width=lambda d: d['prophet_upper'] - d['prophet_lower'])
              .pivot(index='hour', columns='weekday', values='ci_width')
              .reindex(columns=weekday_order))
sns.heatmap(heatmap_ci, cmap='PuBuGn', annot=True, fmt='.1f',
            cbar_kws={'label': 'CI Width (90%)'}, linewidths=0.5, ax=axes[1])
axes[1].set_title("Prophet 90% CI Width (Hourly)", fontsize=13, fontweight='bold')
axes[1].set_xlabel("Weekday"); axes[1].set_ylabel("Hour (UTC)")

plt.tight_layout()
plt.savefig('prophet_heatmaps_improved.png', dpi=150, bbox_inches='tight')
plt.show(); plt.close()

# Per-weekday line charts with CI fill
fig, axes = plt.subplots(4, 2, figsize=(18, 24)); axes = axes.flatten()
for idx, wd in enumerate(weekday_order):
    ax = axes[idx]
    sub = prophet_agg[prophet_agg['weekday'] == wd].sort_values('hour')
    ax.fill_between(sub['hour'], sub['prophet_lower'], sub['prophet_upper'], 
                     alpha=0.2, color='purple', label='Prophet 90% CI')
    ax.plot(sub['hour'], sub['prophet_mean'], 'o-', color='purple', 
            linewidth=2.5, markersize=6, label='Prophet Forecast')
    ax.set_title(f'{wd} — Prophet Forecast', fontsize=12, fontweight='bold')
    ax.set_xlabel('Hour (UTC)'); ax.set_ylabel('Sessions/Hour')
    ax.legend(loc='upper left', fontsize=8)
    ax.grid(alpha=0.3, linestyle='--')
    ax.set_xticks(range(0,24,2)); ax.set_xlim(-0.5, 23.5)
axes[7].axis('off')
plt.tight_layout()
plt.savefig('prophet_weekday_lines_improved.png', dpi=150, bbox_inches='tight')
plt.show(); plt.close()

# Daily totals bar chart
x = np.arange(len(weekday_order)); width = 0.6
fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(x, daily_totals.values, width, color='purple', alpha=0.85, 
              edgecolor='black', linewidth=1, label='Prophet')
ax.set_xticks(x); ax.set_xticklabels(weekday_order, fontsize=11)
ax.set_ylabel('Total Daily Sessions'); ax.set_title('Daily Totals — Prophet Forecast', 
                                                    fontsize=13, fontweight='bold')
ax.grid(axis='y', alpha=0.3, linestyle='--'); ax.legend()
for bar in bars:
    h = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, h + max(daily_totals)*0.01, 
            f'{h:.0f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
plt.tight_layout()
plt.savefig('prophet_daily_totals_improved.png', dpi=150, bbox_inches='tight')
plt.show(); plt.close()

print("\n✅ Static visualizations saved")

## 9️⃣ Prophet Components & Diagnostics

In [ ]:
print("\n" + "="*70)
print("PROPHET COMPONENTS & DIAGNOSTICS")
print("="*70)

# Components plot
fig_components = model_full.plot_components(forecast)
plt.savefig('prophet_components_improved.png', dpi=150, bbox_inches='tight')
plt.show(); plt.close()

# Diagnostics on training set
print("\n📊 Creating model diagnostics summary plot...")
_in_sample_fc = model_full.predict(df_full[['ds']])
_in = df_full.merge(_in_sample_fc[['ds','yhat','yhat_lower','yhat_upper']], on='ds', how='left')
_in['residual'] = _in['y'] - _in['yhat']
coverage_train = float(((_in['y'] >= _in['yhat_lower']) & (_in['y'] <= _in['yhat_upper'])).mean())
print(f"   Training set coverage @90%: {coverage_train:.3f}")

fig = plt.figure(figsize=(18, 12))

# 1) Residuals over time
ax1 = fig.add_subplot(2,2,1)
ax1.plot(_in['ds'], _in['residual'], linewidth=1, alpha=0.6)
ax1.axhline(0, color='red', linewidth=2, linestyle='--')
ax1.set_title('Residuals Over Time', fontsize=12, fontweight='bold')
ax1.set_xlabel('Date'); ax1.set_ylabel('Residual (y - yhat)')
ax1.grid(alpha=0.3, linestyle='--')

# 2) Residuals vs Fitted
ax2 = fig.add_subplot(2,2,2)
ax2.scatter(_in['yhat'], _in['residual'], s=10, alpha=0.4)
ax2.axhline(0, color='red', linewidth=2, linestyle='--')
ax2.set_title('Residuals vs Fitted', fontsize=12, fontweight='bold')
ax2.set_xlabel('Fitted (yhat)'); ax2.set_ylabel('Residual')
ax2.grid(alpha=0.3, linestyle='--')

# 3) Histogram of Residuals
ax3 = fig.add_subplot(2,2,3)
ax3.hist(_in['residual'].dropna(), bins=50, alpha=0.7, edgecolor='black')
ax3.axvline(0, color='red', linewidth=2, linestyle='--')
ax3.set_title('Residuals Histogram', fontsize=12, fontweight='bold')
ax3.set_xlabel('Residual'); ax3.set_ylabel('Frequency')
ax3.grid(alpha=0.3, linestyle='--')

# 4) Q-Q Plot
ax4 = fig.add_subplot(2,2,4)
(_osm, _osr), (_slope, _intercept, _r) = _stats.probplot(_in['residual'].dropna(), dist="norm")
ax4.scatter(_osm, _osr, s=10, alpha=0.6)
ax4.plot(_osm, _slope*_osm + _intercept, 'r-', linewidth=2)
ax4.set_title('Q-Q Plot of Residuals (Normal)', fontsize=12, fontweight='bold')
ax4.set_xlabel('Theoretical Quantiles'); ax4.set_ylabel('Ordered Residuals')
ax4.grid(alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('prophet_diagnostics_improved.png', dpi=150, bbox_inches='tight')
plt.show(); plt.close()

print("\n✅ Diagnostics plots saved")

## 🔟 Summary Report

In [ ]:
print("\n" + "="*70)
print("SUMMARY REPORT")
print("="*70)

print("\n📊 Dataset:")
print(f"   Training period: {TRAINING_MONTHS} months")
print(f"   Total hours: {len(df_prophet)}")
print(f"   Date range: {df_prophet['ds'].min()} to {df_prophet['ds'].max()}")
print(f"   Train/Test split: {int((1-TEST_SIZE)*100)}/{int(TEST_SIZE*100)}%")

print("\n🤖 Model Configuration:")
print(f"   Growth: {model_full.growth}")
print(f"   Changepoint prior: {model_full.changepoint_prior_scale}")
print(f"   Seasonality prior: {model_full.seasonality_prior_scale}")
print(f"   Seasonality mode: {model_full.seasonality_mode}")
print(f"   Seasonalities: Daily={model_full.daily_seasonality}, Weekly={model_full.weekly_seasonality}, Yearly={model_full.yearly_seasonality}")

print("\n📈 Performance Metrics (Test Set):")
print(f"   RMSE: {rmse:.2f} sessions/hour")
print(f"   MAE: {mae:.2f} sessions/hour")
print(f"   MAPE: {mape:.2f}%")
print(f"   R²: {r2:.3f}")
print(f"   Coverage: {coverage:.1%}")

print("\n🔮 Forecast Summary (Next {} Days):".format(FORECAST_DAYS))
print(f"   Avg hourly: {prophet_agg['prophet_mean'].mean():.2f} sessions/hour")
print(f"   Total period: {prophet_agg['prophet_mean'].sum():.0f} sessions")
print(f"   Peak hour: {prophet_agg['prophet_mean'].max():.2f} sessions/hour")
print(f"   Min hour: {prophet_agg['prophet_mean'].min():.2f} sessions/hour")

print("\n✅ Analysis complete!")

In [ ]:
# 📋 View Tuning Results & Best Model

print("\n" + "="*70)
print("MODEL INFORMATION")
print("="*70)

# Check if tuning was performed
if RUN_TUNING and 'tuning_results' in locals():
    print("\n✅ Tuning results available!")
    print("\n📊 Top 10 Models by RMSE:")
    cols = ['changepoint_prior_scale', 'seasonality_prior_scale', 'seasonality_mode', 
            'rmse', 'mae', 'mape', 'coverage']
    print(tuning_results[cols].head(10).to_string(index=False))
    
    print("\n📊 Top 10 Models by MAPE:")
    print(tuning_results[cols].sort_values('mape').head(10).to_string(index=False))
else:
    print("\nℹ️  Tuning was not run (RUN_TUNING=False)")
    print("   Using default parameters shown below.")
    print("   To enable tuning: Set RUN_TUNING=True in configuration cell and re-run.")

# Current model parameters (always available)
print("\n🤖 Current Model Parameters:")
print(f"   Growth: {model_full.growth}")
print(f"   Changepoint Prior Scale: {model_full.changepoint_prior_scale}")
print(f"   Seasonality Prior Scale: {model_full.seasonality_prior_scale}")
print(f"   Seasonality Mode: {model_full.seasonality_mode}")
print(f"   Daily Seasonality: {model_full.daily_seasonality}")
print(f"   Weekly Seasonality: {model_full.weekly_seasonality}")
print(f"   Yearly Seasonality: {model_full.yearly_seasonality}")
print(f"   Interval Width: {model_full.interval_width}")

# Model structure
print("\n📈 Model Structure:")
print(f"   Changepoints detected: {len(model_full.changepoints)}")
if len(model_full.changepoints) > 0:
    print(f"   First changepoint: {model_full.changepoints.iloc[0] if hasattr(model_full.changepoints, 'iloc') else model_full.changepoints[0]}")
    print(f"   Last changepoint: {model_full.changepoints.iloc[-1] if hasattr(model_full.changepoints, 'iloc') else model_full.changepoints[-1]}")
print(f"   Training observations: {len(df_full)}")

print("\n✅ Model information displayed successfully!")


## 🎨 Interactive Visualizations (Plotly)

In [ ]:
print("\n" + "="*70)
print("INTERACTIVE VISUALIZATIONS (PLOTLY)")
print("="*70)

# 1. Interactive Time Series with Forecast
print("\n📊 Creating interactive forecast plot...")

# Prepare data for plotting
historical = df_full.copy()
historical['type'] = 'Historical'

forecast_plot = forecast_future.copy()
forecast_plot['type'] = 'Forecast'
forecast_plot = forecast_plot.rename(columns={'yhat': 'y'})

# Create figure
fig1 = go.Figure()

# Historical data
fig1.add_trace(go.Scatter(
    x=historical['ds'],
    y=historical['y'],
    mode='lines',
    name='Historical Data',
    line=dict(color='#1f77b4', width=2),
    hovertemplate='<b>Date:</b> %{x}<br><b>Sessions:</b> %{y:.1f}<extra></extra>'
))

# Forecast line
fig1.add_trace(go.Scatter(
    x=forecast_plot['ds'],
    y=forecast_plot['y'],
    mode='lines',
    name='Forecast',
    line=dict(color='#ff7f0e', width=3, dash='dash'),
    hovertemplate='<b>Date:</b> %{x}<br><b>Forecast:</b> %{y:.1f}<extra></extra>'
))

# Confidence interval
fig1.add_trace(go.Scatter(
    x=forecast_plot['ds'],
    y=forecast_plot['yhat_upper'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

fig1.add_trace(go.Scatter(
    x=forecast_plot['ds'],
    y=forecast_plot['yhat_lower'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(255, 127, 14, 0.2)',
    fill='tonexty',
    name='90% Confidence Interval',
    hovertemplate='<b>Date:</b> %{x}<br><b>Lower:</b> %{y:.1f}<extra></extra>'
))

fig1.update_layout(
    title=f'Hourly Web Traffic Forecast ({FORECAST_DAYS} Days Ahead)',
    xaxis_title='Date',
    yaxis_title='Sessions/Hour',
    hovermode='x unified',
    height=500,
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig1.show()

# 2. Interactive Heatmap - Forecast by Day/Hour
print("\n📊 Creating interactive heatmap...")

heatmap_data = prophet_agg.pivot(index='hour', columns='weekday', values='prophet_mean').reindex(columns=weekday_order)

fig2 = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='YlOrRd',
    hovertemplate='<b>%{x}</b><br>Hour: %{y}<br>Sessions: %{z:.1f}<extra></extra>',
    colorbar=dict(title='Sessions/Hour')
))

fig2.update_layout(
    title='Forecast Heatmap: Sessions by Day & Hour',
    xaxis_title='Day of Week',
    yaxis_title='Hour (UTC)',
    height=600,
    template='plotly_white'
)

fig2.update_yaxes(autorange='reversed')

fig2.show()

# 3. Interactive Daily Totals Bar Chart
print("\n📊 Creating interactive bar chart...")

daily_totals_df = pd.DataFrame({
    'weekday': weekday_order,
    'total': daily_totals.reindex(weekday_order).values
})

fig3 = go.Figure(data=[
    go.Bar(
        x=daily_totals_df['weekday'],
        y=daily_totals_df['total'],
        marker_color='purple',
        marker_line_color='black',
        marker_line_width=1.5,
        hovertemplate='<b>%{x}</b><br>Total Sessions: %{y:.0f}<extra></extra>',
        text=daily_totals_df['total'].round(0),
        textposition='outside',
        texttemplate='%{text:.0f}'
    )
])

fig3.update_layout(
    title='Daily Total Sessions by Weekday (Forecast)',
    xaxis_title='Day of Week',
    yaxis_title='Total Daily Sessions',
    height=500,
    template='plotly_white',
    showlegend=False
)

fig3.show()

# 4. Interactive Components Decomposition (Weekly, Yearly, Daily)
print("\n📊 Creating interactive components plot...")

# Extract components from forecast
components_df = forecast[['ds', 'weekly', 'yearly', 'daily']].tail(FORECAST_HOURS * 2)  # Last 2x forecast period

fig4 = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Weekly Seasonality', 'Yearly Seasonality', 'Daily Seasonality'),
    vertical_spacing=0.1,
    row_heights=[0.33, 0.33, 0.33]
)

# Weekly
fig4.add_trace(
    go.Scatter(x=components_df['ds'], y=components_df['weekly'], 
               mode='lines', name='Weekly', line=dict(color='green', width=2),
               hovertemplate='<b>Date:</b> %{x}<br><b>Weekly:</b> %{y:.1f}<extra></extra>'),
    row=1, col=1
)

# Yearly
fig4.add_trace(
    go.Scatter(x=components_df['ds'], y=components_df['yearly'], 
               mode='lines', name='Yearly', line=dict(color='red', width=2),
               hovertemplate='<b>Date:</b> %{x}<br><b>Yearly:</b> %{y:.1f}<extra></extra>'),
    row=2, col=1
)

# Daily
fig4.add_trace(
    go.Scatter(x=components_df['ds'], y=components_df['daily'], 
               mode='lines', name='Daily', line=dict(color='purple', width=2),
               hovertemplate='<b>Date:</b> %{x}<br><b>Daily:</b> %{y:.1f}<extra></extra>'),
    row=3, col=1
)

fig4.update_xaxes(title_text="Date", row=3, col=1)
fig4.update_yaxes(title_text="Effect", row=1, col=1)
fig4.update_yaxes(title_text="Effect", row=2, col=1)
fig4.update_yaxes(title_text="Effect", row=3, col=1)

fig4.update_layout(
    title_text='Forecast Components Decomposition',
    height=800,
    template='plotly_white',
    showlegend=False
)

fig4.show()

# 5. Interactive Residuals Analysis (if test data available)
print("\n📊 Creating interactive residuals plot...")

if 'df_test' in locals() and 'forecast_test' in locals() and len(df_test) > 0:
    residuals_df = pd.DataFrame({
        'ds': df_test['ds'].values,
        'actual': df_test['y'].values,
        'predicted': forecast_test['yhat'].values,
        'residual': df_test['y'].values - forecast_test['yhat'].values
    })
    
    fig5 = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Residuals Over Time', 'Residuals vs Predicted', 
                        'Residual Distribution', 'Actual vs Predicted'),
        specs=[[{"type": "scatter"}, {"type": "scatter"}],
               [{"type": "histogram"}, {"type": "scatter"}]],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # Residuals over time
    fig5.add_trace(
        go.Scatter(x=residuals_df['ds'], y=residuals_df['residual'],
                   mode='markers', name='Residuals',
                   marker=dict(size=4, color='blue', opacity=0.6),
                   hovertemplate='<b>Date:</b> %{x}<br><b>Residual:</b> %{y:.1f}<extra></extra>'),
        row=1, col=1
    )
    fig5.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=1)
    
    # Residuals vs Predicted
    fig5.add_trace(
        go.Scatter(x=residuals_df['predicted'], y=residuals_df['residual'],
                   mode='markers', name='Residuals',
                   marker=dict(size=4, color='green', opacity=0.6),
                   hovertemplate='<b>Predicted:</b> %{x:.1f}<br><b>Residual:</b> %{y:.1f}<extra></extra>'),
        row=1, col=2
    )
    fig5.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=2)
    
    # Residual distribution
    fig5.add_trace(
        go.Histogram(x=residuals_df['residual'], nbinsx=30,
                     marker_color='purple', opacity=0.7,
                     name='Distribution',
                     hovertemplate='<b>Residual Range:</b> %{x}<br><b>Count:</b> %{y}<extra></extra>'),
        row=2, col=1
    )
    
    # Actual vs Predicted
    fig5.add_trace(
        go.Scatter(x=residuals_df['actual'], y=residuals_df['predicted'],
                   mode='markers', name='Predictions',
                   marker=dict(size=4, color='orange', opacity=0.6),
                   hovertemplate='<b>Actual:</b> %{x:.1f}<br><b>Predicted:</b> %{y:.1f}<extra></extra>'),
        row=2, col=2
    )
    # Perfect prediction line
    min_val = min(residuals_df['actual'].min(), residuals_df['predicted'].min())
    max_val = max(residuals_df['actual'].max(), residuals_df['predicted'].max())
    fig5.add_trace(
        go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                   mode='lines', name='Perfect Fit',
                   line=dict(color='red', dash='dash', width=2),
                   hoverinfo='skip'),
        row=2, col=2
    )
    
    fig5.update_xaxes(title_text="Date", row=1, col=1)
    fig5.update_xaxes(title_text="Predicted Value", row=1, col=2)
    fig5.update_xaxes(title_text="Residual", row=2, col=1)
    fig5.update_xaxes(title_text="Actual Value", row=2, col=2)
    
    fig5.update_yaxes(title_text="Residual", row=1, col=1)
    fig5.update_yaxes(title_text="Residual", row=1, col=2)
    fig5.update_yaxes(title_text="Count", row=2, col=1)
    fig5.update_yaxes(title_text="Predicted Value", row=2, col=2)
    
    fig5.update_layout(
        title_text='Residuals Analysis - Model Diagnostics',
        height=800,
        template='plotly_white',
        showlegend=False
    )
    
    fig5.show()
    print("✅ Residuals plot created successfully!")
else:
    print("⚠️  Skipping residuals plot - test data or forecast_test not available")

print("\n✅ All interactive visualizations created successfully!")
print("   💡 Tip: Hover over plots for details, click and drag to zoom, double-click to reset")
